# Prototype 7: Using CoastSat to download ROIs
---
- Author: Sharon Fitzpatrick
- Date: 1/28/2022

## Description
This prototype showcases an interactive map where users can click on various ROIs to download their data with CoastSat.It also included feedback from prototype 6 and some improvments to previous functions.

## The Workflow
1. The user draws a bounding box
2. The user runs the code to clip the coastline geojson to the bounding box
3. The user runs the code to add the coastline within the bounding box to the map. (The coastline will be in yellow)
4. The user runs the code to to generate polygons along the portion of the coastline vector within the bounding box
5. The user runs the code to plot the polygons and associated markers to the map.
6. The user selects at least one ROI for download with CoastSat by clicking on any ROI on the map
7. The user runs the code to download the geojson for the ROIs they selected
8. The user runs the code to download all the data using CoastSat for the ROIs they selected.

## Improvements & Features
- Adjusted the ```ROI size=0.04``` in **create_rectangles()** 
- Ajusted the ```MAX_BBOX_SIZE=3000000000``` in **check_bbox_size**
- Created unique sitenames for ROI's dataset downloaded from CoastSat

## Bug Fixes
1. The bug where closed shapes such as islands would not download properly has been fixed
2. The bug where unauthenicated users would recieve an exception when checking for available images from CoastSat has been patched

# Get the Bounding Box Geojson
---
## Directions
1. Use the square drawing button to draw a bounding box somewhere along the coast

## !! Warning !!
- If the bounding box drawn is too small or too large you will need to re-run the code to generate the map again.

In [93]:
from ipywidgets import Layout
map_settings={
"center_point":( 36.100206171202295, -75.71024865851251),
"zoom":13,
 "draw_control":False,
 "measure_control":False, 
 "fullscreen_control":False, 
 "attribution_control":True,
 "Layout":Layout(width='100%', height='100px')
}

In [94]:
from ipyleaflet import DrawControl, Map, GeoJSON, Polygon
import leafmap
from shapely.geometry import shape, LineString, Point
from shapely.geometry import Point as shapely_Point
from area import area
import geopandas as gpd
import json
import os
from shapely.ops import unary_union
import numpy as np
from geojson import Point, Feature, FeatureCollection, dump

In [95]:
# Empty list to hold all the polygons drawn by the user
shapes_list=[]

# Disable polyline, circle, and rectangle 
m = leafmap.Map(draw_control=map_settings["draw_control"],
                measure_control=map_settings["measure_control"],
                fullscreen_control=map_settings["fullscreen_control"],
                attribution_control=map_settings["attribution_control"],
                center=map_settings["center_point"],
                zoom=map_settings["zoom"],
                layout=map_settings["Layout"])

draw_control = DrawControl()

draw_control.polyline = {}
draw_control.circlemarker = {}
# Custom styles for polygons and rectangles
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "green",
        "color": "green",
        "fillOpacity": 0.2,
        "Opacity": 0.2
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Ops!"
    },
    "allowIntersection": False,
    "transform":True
}

draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "green",
        "color": "green",
        "fillOpacity": 0.1,
        "Opacity": 0.1
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Ops!"
    },
    "allowIntersection": False,
    "transform":True
}

# Each time a polygon is drawn it is appended to the shapeslist which is used to create the bounding box
def handle_draw(target, action, geo_json):
    if draw_control.last_action == 'created'and draw_control.last_draw['geometry']['type']=='Polygon' :
        shapes_list.append( draw_control.last_draw['geometry'])
    if draw_control.last_action == 'deleted':
        label1.value=str(f"{draw_control.last_draw['geometry']['type']}  {draw_control.last_action}")
        shapes_list.pop()

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

m


Map(center=[36.100206171202295, -75.71024865851251], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Select the name of the file that will hold the geojson for all the ROIs generated

In [96]:
filename = "roi.geojson"

# Check the size of the Bounding Box 
1. Obtain the area of the bounding box using **calculate_area_bbox**
3. Check if its a valid size suing **check_bbox_size**

### Custom Exceptions for Invalid BBox size
1. BboxTooLargeError
2. BboxTooSmallError

In [97]:
class BboxTooLargeError(Exception):
    """BboxTooLargeError: raised when bounding box is larger than MAX_BBOX_SIZE
    Args:
        Exception: Inherits from the base exception class
    """
    def __init__(self, msg="The bounding box was too large."):
        self.msg=msg
        super().__init__(self.msg)
    def __str__(self):
        return (f"{self.msg}")
    
class BboxTooSmallError(Exception):
    """BboxTooLargeError: raised when bounding box is smaller than MIN_BBOX_SIZE
    Args:
        Exception: Inherits from the base exception class
    """
    def __init__(self, msg="The bounding box was too small."):
        self.msg=msg
        super().__init__(self.msg)
    def __str__(self):
        return (f"{self.msg}")

In [98]:
def calculate_area_bbox(bbox):
    bbox_area=round(area(bbox),2)
    print(f"Area: {bbox_area} meters squared")
    return bbox_area

## Function erase_bbox
- Clears any data associated with invalid bbox
- NOTE: If the bbox size is invalid re-run the code to re-create the map


In [99]:
def erase_bbox(draw_control : 'ipyleaflet.leaflet.DrawControl',shapeslist:list)-> list:
    """Erases bounding box from map and deletes the contents of the shapeslist"""
    draw_control.clear()
    shapeslist=[]
    return shapeslist

## Check if the bounding box size is allowed

In [100]:
def check_bbox_size(bbox_area:float,shapes_list:list):
    """"Raises an exception and clears the map if the size of the bounding box is too large or small."""
    # CONSTANT SIZE VARIABLES
    # UNITS = Sq. Meters
    MAX_BBOX_SIZE=3000000000
    MIN_BBOX_SIZE=300
    # Check if the size is greater than MAX_BBOX_SIZE
    if bbox_area > MAX_BBOX_SIZE:
        shapes_list=erase_bbox(draw_control,shapes_list)
        raise BboxTooLargeError
    # Check if size smaller than MIN_BBOX_SIZE
    elif bbox_area < MIN_BBOX_SIZE:
        shapes_list=erase_bbox(draw_control,shapes_list)
        raise BboxTooSmallError

In [101]:
if not shapes_list == []:
    last_index=len(shapes_list)-1
#     The last index in the shapes_list was the last shape drawn
    bbox_area=calculate_area_bbox(shapes_list[last_index])
    check_bbox_size(bbox_area,shapes_list)
else:
     print("ERROR.\nYou must draw a bounding box somewhere on the coast first.")

Area: 204928550.4 meters squared


# Clip the Bounding Box and Vector
---
1. Convert the bounding box to a geopandas geodataframe using **create_geodataframe_from_bbox**
2. Print the bounds of the geodataframe

In [102]:
def create_geodataframe_from_bbox(shapes_list:list)->"geopandas.geodataframe.GeoDataFrame":
    """
    Create a geodataframe from the bounding box's geometry
    """
    if not shapes_list == []:
        geom = [shape(i) for i in shapes_list]
        geojson_bbox=gpd.GeoDataFrame({'geometry':geom})
        geojson_bbox.crs='EPSG:4326'
        geojson_bbox.plot()
        return geojson_bbox
    else:
        print("ERROR.\nYou must draw a bounding box somewhere on the coast first.")
        return None

Geojson bounds
        minx       miny        maxx       maxy
0 -120.47169  33.994988 -120.271243  34.094555



In [103]:
geojson_bbox=create_geodataframe_from_bbox(shapes_list)
if (geojson_bbox is not None):
    print(f"Geojson bounds\n{geojson_bbox.bounds}\n")
else:
    print("ERROR.\nYou must draw a bounding box somewhere on the coast first.")

### Download the Shorline Vector
- https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196

In [104]:
def get_shoreline_vector(filename:str)->"geopandas.geodataframe.GeoDataFrame":
    """
    Read the coastline's geojson into a geodataframe from the file named filename
    """
    if os.path.exists(filename):
         with open(filename, 'r') as f:
            coastline_vector=gpd.read_file(f)
    else:
        print('File does not exist. Please download the coastline_vector necessary here: https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196 ')
        raise FileNotFoundError
    return coastline_vector      

In [105]:
coastline_vector=get_shoreline_vector("stanford-xv279yj9196.geojson")
coastline_vector

,id,f_code,acc,exs,soc,geometry
0,xv279yj9196.1,BA010,1,44,USA,"MULTILINESTRING ((-96.93882 28.02606, -96.9354..."
1,xv279yj9196.2,BA010,1,44,USA,"MULTILINESTRING ((-96.91022 28.06370, -96.9123..."
2,xv279yj9196.3,BA010,1,44,USA,"MULTILINESTRING ((-96.89713 28.06679, -96.8990..."
3,xv279yj9196.4,BA010,1,44,USA,"MULTILINESTRING ((-96.88841 28.07634, -96.8849..."
4,xv279yj9196.5,BA010,1,44,USA,"MULTILINESTRING ((-96.91283 28.12183, -96.9144..."
...,...,...,...,...,...,...
3094,xv279yj9196.3095,BA010,1,44,USA,"MULTILINESTRING ((-160.08739 22.02016, -160.08..."
3095,xv279yj9196.3096,BA010,1,44,USA,"MULTILINESTRING ((-159.39618 22.22735, -159.39..."
3096,xv279yj9196.3097,BA010,1,44,USA,"MULTILINESTRING ((-130.01769 55.91194, -130.02..."
3097,xv279yj9196.3098,BA010,1,44,USA,"MULTILINESTRING ((-160.87141 56.00000, -160.86..."


### Clip the shorline vector to the bounding box

In [106]:
def clip_coastline_to_bbox(coastline_vector: "geopandas.geodataframe.GeoDataFrame", geojson_bbox: "geopandas.geodataframe.GeoDataFrame")->"geopandas.geodataframe.GeoDataFrame":
    """
    Clips the geojson coastline_vector within geojson_bbox,the geojson bounding box.
    
     Arguments:
    -----------
    coastline_vector: geopandas.geodataframe.GeoDataFrame
        GeoDataFrame contains the coastline's geometry
        
    geojson_bbox: "geopandas.geodataframe.GeoDataFrame"
        GeoDataFrame containing the bounding box geometry
    Returns
    --------
    roi_coast : "geopandas.geodataframe.GeoDataFrame"
        roi_coast a GeoDataFrame that holds the clipped portion of the coastline_vector within geojson_bbox
    
    """
    #clip coastal polyline
    roi_coast=gpd.clip(coastline_vector, geojson_bbox)
    roi_coast=roi_coast.to_crs('EPSG:4326')
    return roi_coast

In [107]:
roi_coast=clip_coastline_to_bbox(coastline_vector, geojson_bbox)
roi_coast

,id,f_code,acc,exs,soc,geometry
614,xv279yj9196.612,BA010,1,44,USA,"LINESTRING (-120.35985 34.05197, -120.35789 34..."
612,xv279yj9196.610,BA010,1,44,USA,"LINESTRING (-120.43544 34.05650, -120.43546 34..."
613,xv279yj9196.611,BA010,1,44,USA,"LINESTRING (-120.33112 34.05641, -120.33279 34..."


## Display the Shoreline Vector

In [108]:
if not roi_coast.empty:
    roi_coast.plot()
else:
    print("ERROR.\nThe bounding box provided did not intersect with the coastline vector provided.")

### Add the coastline vector within the bounding box to the map

In [109]:
vector_within_bounding_box_json=roi_coast.to_json()
# to_json generates a string that is json format
res = json.loads(vector_within_bounding_box_json)
geo_json_vector = GeoJSON(
    data=res,
    style={
        'color':'yellow','fill_color':'yellow' ,'opacity': 1, 'dashArray': '5', 'fillOpacity': 0.5, 'weight': 4
    },
    hover_style={
        'color': 'white', 'dashArray': '4', 'fillOpacity': 0.7
    },
)
m.add_layer(geo_json_vector)
m

Map(bottom=419115.18173217773, center=[34.016474746300055, -120.31488352768683], controls=(ZoomControl(options…

# Generate Polygons along the selected portion of the coastline vector
---

### Convert the clipped portion of the coastline into geojson

In [110]:
# vector_within_bounding_box_geojson
vector_within_bounding_box_geojson = json.loads(vector_within_bounding_box_json)

In [111]:
print(f"There are {len(vector_within_bounding_box_geojson['features'])} feature(s) in the bounding box")

There are 3 feature(s) in the bounding box


# Convert the geometry of coastline within the bounding box to a list of linestrings
For each of the linestrings and the linestrings within the multilinestring coastline that make up the coastline vector add them to the list of linestrings. These linestrings will have pointers interpolated along them later

In [112]:
def create_linestring_list(vector_within_bounding_box_geojson:dict)->list:
    """
    Create a list of linestrings from the multilinestrings and linestrings that compose the vector

    Arguments:
    -----------
    vector_within_bounding_box_geojson: dict
        geojson vector 
    Returns:
    -----------
    lines_list: list
        list of multiple shapely.geometry.linestring.LineString that represent each segment of the vector
    """
    lines_list=[]
    length_vector_bbox_features=len(vector_within_bounding_box_geojson['features'])
    length_vector_bbox_features

    if(length_vector_bbox_features != 1):
        for i in range(0,length_vector_bbox_features):
            if  vector_within_bounding_box_geojson['features'][i]['geometry']['type'] == 'MultiLineString':
                for y in range(len(vector_within_bounding_box_geojson['features'][i]['geometry']['coordinates'])):
                    line=LineString(vector_within_bounding_box_geojson['features'][i]['geometry']['coordinates'][y])
                    lines_list.append(line)
            elif  vector_within_bounding_box_geojson['features'][i]['geometry']['type'] == 'LineString':
                line=LineString(vector_within_bounding_box_geojson['features'][i]['geometry']['coordinates'])
                lines_list.append(line)
    else:
        for i in range(0,len(vector_within_bounding_box_geojson['features'])):
            if  vector_within_bounding_box_geojson['features'][0]['geometry']['type'] == 'MultiLineString':
                for y in range(len(vector_within_bounding_box_geojson['features'][0]['geometry']['coordinates'])):
                    line=LineString(vector_within_bounding_box_geojson['features'][0]['geometry']['coordinates'][y])
                    lines_list.append(line)
            elif  vector_within_bounding_box_geojson['features'][i]['geometry']['type'] == 'LineString':
                line=LineString(vector_within_bounding_box_geojson['features'][i]['geometry']['coordinates'])
                lines_list.append(line)
    return lines_list

In [113]:
lines_list=create_linestring_list(vector_within_bounding_box_geojson)

In [114]:
print(f"The number of linestring(s) in lines_list: {len(lines_list)}")

The number of linestring(s) in lines_list: 3


## Interpolate Equal Distances Between Each Point on the Vector
For each LineString that makes up the coastline :
1. Calculate the distance between each point so that they are equally spaced
2. Create a tuple of each of the equally spaced distance points
3. Check if the linestring is closed and if it is then choose the 1st coordinate as the boundary
4. Interpolate the linestring for each of the distances points calculated earlier
5. Add the new multipoint generated by the interpolation to multipoint_list

In [115]:
def interpolate_points(lines_list:list,num_pts=10)->list:
    """
    Create a list of multipoints for the interpolated points along each line in the lines_list

    Arguments:
    -----------
    lines_list: list
        list of multiple shapely.geometry.linestring.LineString that represent each segment of the vector
    
    num_pts: int
        integer value representing the number of interpolated points created per LineString
        
    Returns:
    -----------
    multipoint_list: list
    A list of multiple shapely.geometry.multipoint.MultiPoint 
    """
    # multipoint_list holds the multipoint for each feature of the coastline within the bbox
    multipoint_list=[]

    for i in range(len(lines_list)):
        line=lines_list[i]
        distance_delta=line.length/num_pts
        distances = np.arange(0, line.length, distance_delta)
        if lines_list[i].is_closed:
            #Its a closed shape so its boundary points are NULL
            boundary=shapely_Point(lines_list[i].coords[0])
            # print("\n Closed: ", boundary, type( boundary))
        else: 
            boundary=lines_list[i].boundary.geoms[0]
            # print("\n NOT closed: ",boundary, type(boundary))
        points = [lines_list[i].interpolate(distance) for distance in distances] + [boundary]
        multipoint = unary_union(points) 
        multipoint
        multipoint_list.append(multipoint)
    return multipoint_list

In [116]:
multipoint_list=interpolate_points(lines_list)

## Convert the multipoint from the interpolation to a tuple
- ipyleaflet requires that the points be a tuple to be plotted onto the map

In [117]:
def convert_multipoints_to_tuples(multipoint_list:list)-> list:
    """
    Create a list of tuples for the points in multipoint_list

    Arguments:
    -----------
    multipoint_list: list
        A list of multiple shapely.geometry.multipoint.MultiPoint
    Returns:
    -----------
    points_list_collection: list
        A list of tuples each tuple represents a single point
    """
    points_list_collection=[]
    for multipoint in  multipoint_list:
        # First get each point from the multipoint object
        points_array=[point for point in multipoint.geoms]
        # Create an empty array to hold all the points as tuples
        points_list=[]
        # For each point swap lat and lng because ipyleaflet swaps them
        for point in points_array:
            point_tuple=(point.coords[0][1],point.coords[0][0])
            points_list.append(point_tuple)

        points_list_collection.append(points_list)
    return points_list_collection

In [118]:
points_list_collection=convert_multipoints_to_tuples(multipoint_list)
points_list_collection

[[(34.03168150386515, -120.45145562605194),
  (34.0474437882485, -120.42865616851695),
  (34.02605157899716, -120.42036365010279),
  (34.05162941609134, -120.39809138700905),
  (34.02471516472867, -120.38497973558093),
  (34.07348445562798, -120.36940888741836),
  (34.051975, -120.3598465),
  (34.01907111842292, -120.35114603642788),
  (34.04466206991077, -120.32408092353609),
  (34.01953461402038, -120.31268576642063)],
 [(34.05426184201174, -120.45201113026535),
  (34.057512237655345, -120.45094122767935),
  (34.05264068476356, -120.44859420156101),
  (34.057930632990455, -120.44698128474226),
  (34.05378232632252, -120.44485544798098),
  (34.05782907975453, -120.44297002003749),
  (34.05458667145645, -120.44117597525123),
  (34.05772752651861, -120.4389587553327),
  (34.05371622783362, -120.43725897569914),
  (34.05650021, -120.43543592)],
 [(34.05858484563296, -120.33828097847295),
  (34.06015860823383, -120.33708348625552),
  (34.05742208720035, -120.33657115772523),
  (34.0611644

## Function to Convert ipyleaflet Polygon Points to GeoJson
1. Ipyleaflet draws its shapes in lat,lng format and it must be converted to lng, lat for geojson.
2. In order to correctly draw a rectangle in geojson the order of the points matters as well.

In [119]:
def convert_to_geojson(upper_right_y : float, upper_right_x: float,upper_left_y: float, upper_left_x: float,lower_left_y: float,  lower_left_x: float,lower_right_y: float,lower_right_x: float) -> dict:
    """Convert the 4 corners of the rectangle into geojson  """
    geojson_feature={}
    geojson_feature["type"]="Feature"
    geojson_feature["properties"]={}
    geojson_feature["geometry"]={}
    
    geojson_polygon={}
    geojson_polygon["type"]="Polygon"
    geojson_polygon["coordinates"]=[]
#     The coordinates(which are 1,2 arrays) are nested within a parent array
    nested_array=[]
    nested_array.append([upper_right_x, upper_right_y])
    nested_array.append([upper_left_x, upper_left_y])
    nested_array.append([lower_left_x, lower_left_y])
    nested_array.append([lower_right_x, lower_right_y])
    #GeoJson rectangles have the first point repeated again as the last point
    nested_array.append([upper_right_x, upper_right_y])

    geojson_polygon["coordinates"].append(nested_array)
    
    geojson_feature["geometry"]=geojson_polygon
    return geojson_feature

## Function to Write Polygons in GeoJson to a .geojson file
1. Pass the filename and the geojson from ***convert_to_geojson()***

In [120]:
def write_to_geojson_file(filename : str,geojson_polygons : dict, perserve_id: bool =False):
    """Make a filename.geojson file from dictionary geojson_polygons 
      
    Arguments:
    -----------
    
    filename : str
        The name of the geojson file to be written to. MUST end in .geojson
    
    geojson_polygons : dict
        The dictionary containing the geojson data. Must contain all geojson within the features section of geojson.
    
    perserve_id: boolean   default=False
        Boolean that if True perserves the property id and writes it to the geojson file
    """
    features = []
    if not perserve_id:
        count=0
        for geoObj in geojson_polygons["features"]:
            features.append(Feature(properties={"id":count },geometry=geoObj["geometry"]))
            count=count+1
    elif perserve_id:
        for geoObj in geojson_polygons["features"]:
            print(geoObj["properties"]["id"])
            features.append(Feature(properties={"id": geoObj["properties"]["id"]},geometry=geoObj["geometry"]))
    
    feature_collection = FeatureCollection(features)
    with open(f'{filename}', 'w') as f:
        dump(feature_collection, f)

## Create a Rectangle For Each Interpolated Point on the Vector
1. Create a list to hold all the polygons to be added on to the map
2. Convert each of the rectangles to geojson for further analysis later using ***convert_to_geojson()***

In [121]:
def create_reactangles(points_list_collection: list,size:int=0.04)-> dict:
    """
    Create the geojson rectangles for each point in the points_list_collection

    Arguments:
    -----------
    points_list_collection: list
        list of tuples containing all the interpolated points along the given vector
    
    size: float
        A float that will be used as the multiplier for the ROI sizes
    
    Returns:
    -----------
    geojson_polygons: dict
       geojson dictionary contains all the rectangles generated
    """                                         
    geojson_polygons={"type": "FeatureCollection","features":[]}

    # Create a rectangle at each point on the line
    # Swap the x and y for each point because ipyleaflet swaps them for draw methods
    for points_list in points_list_collection:
        for point in points_list:
            upper_right_x=point[0]-(size/2)
            upper_right_y=point[1]-(size/2)
            upper_left_x=point[0]+(size/2)
            upper_left_y=point[1]-(size/2)
            lower_left_x=point[0]+(size/2)
            lower_left_y=point[1]+(size/2)
            lower_right_x=point[0]-(size/2)
            lower_right_y=point[1]+(size/2)
            #Convert each set of points to geojson (DONT swap x and y this time)
            geojson_polygon=convert_to_geojson(upper_right_x, upper_right_y,upper_left_x, upper_left_y,lower_left_x,lower_left_y,lower_right_x,lower_right_y)
            geojson_polygons["features"].append(geojson_polygon)
    return geojson_polygons

### Create the ROIs along the coastline and write their data to roi.geojson

In [122]:
geojson_polygons=create_reactangles(points_list_collection)

In [123]:
write_to_geojson_file(filename,geojson_polygons)

## Check if all the ROIs overlap with their neighbors

In [124]:
def check_shapes_overlap(filename):
    """
    Returns True if all the ROIs in the filename's geojson overlap and false if even one ROI does not overlap.
    
    Arguments:
    -----------
    filename: str
        The name of the file containing all the geojson data for all the ROIs generated.
    Returns:
    -----------
    shapes_overlap: boolean
        A boolean that is True only is ALL the ROIs overlap.
    """
    geo_series=gpd.GeoSeries.from_file(filename)
    shapes_overlap=True
    last_index=geo_series.index[-1]
    for index in geo_series.index:
        geo_series_next=geo_series[index]
        for inner_index in geo_series.index:
            if inner_index==last_index and not geo_series_next.overlaps(geo_series[inner_index]) :
                shapes_overlap=False
            if geo_series_next.overlaps(geo_series[inner_index]) and index != inner_index:
                break
    return shapes_overlap

In [125]:
def create_overlap(filename: str, lines_list: list, multipoint_list: list, points_list_collection: list ,geojson_polygons: dict):
    """
    Check if all the ROI overlap with at both of its neighbors. If it doesn't increase the number of ROIs up to 160. If the number of ROIs exceed 160,
    then no more will be drawn even if there is not enough overlap.

    Arguments:
    -----------
    filename: str
        The name of the file containing all the geojson data for all the ROIs generated.
    
    lines_list: list
        list of multiple shapely.geometry.linestring.LineString that represent each segment of the vector
        
   multipoint_list: list
        A list of multiple shapely.geometry.multipoint.MultiPoint
    
    points_list_collection: list
        list of tuples containing all the interpolated points along the given vector

    geojson_polygons: dict
       geojson dictionary contains all the rectangles generated
    
    Returns:
    -----------
    None
    """           
    num_pts=20
    too_many_ROI=False
    all_shapes_overlap=check_shapes_overlap(filename)
    # Convert to while loop when test is done
    while all_shapes_overlap==False and too_many_ROI==False:
        multipoint_list=interpolate_points(lines_list,num_pts)
        points_list_collection=convert_multipoints_to_tuples(multipoint_list)
        geojson_polygons=create_reactangles(points_list_collection)
        write_to_geojson_file(filename,geojson_polygons)
        all_shapes_overlap=check_shapes_overlap(filename)
        if not all_shapes_overlap:
            if num_pts == 100:
                too_many_ROI=True
            elif num_pts > 100:
                num_pts=100
            else:
                num_pts=num_pts*2
        print(f"num_pts increased to: {num_pts}")  

In [126]:
create_overlap(filename, lines_list, multipoint_list, points_list_collection ,geojson_polygons)

# Add the Features to the Map
---
### Markers
    1. Each marker has the lat and lng on hover and on click as a popup
### ROI
    1. Each rectangle can be click on to select it for download with data
    2. Click the rectangle again to unselect it

In [127]:
import copy
from ipyleaflet import LayerGroup
from ipyleaflet import Map, Marker, MarkerCluster
from ipywidgets import HTML
import geojson

# @TODO    Add Try Except later
# Raise excecption for invalid file type
# Raise exception for invalid file format
    # geojson layer with hover handler
with open(filename) as f:
    data = geojson.load(f)

#     Add style to each feature in the geojson
for feature in data["features"]:
    feature["properties"]["style"] = {
        "color": "grey",
        "weight": 1,
         "fillColor": "grey",
        "fillOpacity": 0.2,
     }

selected_set = set()
selected_layer = None


def convert_selected_set_to_geojson(selected_set):
#     Create a geojson feature collection with no features
    geojson = {"type": "FeatureCollection", "features": []}
# Iterate through all the features in the geojson data and check if its name is in the selected_set
# if its name is then add it as a geojson feature
    geojson["features"] = [
        feature
        for feature in data["features"]
        if feature["properties"]["id"] in selected_set
    ]
 #     change the style for the selected features
    for feature in data["features"]:
        feature["properties"]["style"] = {
            "color": "blue",
            "weight": 2,
             "fillColor": "grey",
             "fillOpacity": 0.2,
          }
    return geojson

# If  properties is none is means there is no name and thus nothing to add
# Modify this for my version
# This function allows for the feature to be removed from the selected portion if it is clicked again after being selected
def selected_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties["id"]
    selected_set.remove(cid)
#     If the selected_layer is included then remove it
    if selected_layer is not None:
         m.remove_layer(selected_layer)
    selected_layer = GeoJSON(
        data=convert_selected_set_to_geojson(selected_set),
        name="Selected ROIs",
        hover_style={"fillColor": "blue"},
    )
    selected_layer.on_click(selected_onclick_handler)
    m.add_layer(selected_layer)

    # selected_onclick_handler mirrors this function
def geojson_onclick_handler(event=None, id=None, properties=None, **args):
    print("Click Registered")
    global selected_layer
#     Custom properties associated with geojson
    if properties is None:
           return
    cid = properties["id"]
#     Add the property to the selected_set (unordered, no duplicates, no index)
    selected_set.add(cid)
#     Remove the previously selected layer from map layer
    if selected_layer is not None:
        m.remove_layer(selected_layer)
# Create a new layer from the selected geojson on the map 
# call convert_selected_set_to_geojson() to style the selected geojson
    selected_layer = GeoJSON(
        data=convert_selected_set_to_geojson(selected_set),
        name="Selected ROIs",
        hover_style={"fillColor": "blue"},
    )
# Add a on_click handler to the selected_layer
    selected_layer.on_click(selected_onclick_handler)
# Add the selected layer to the map
    m.add_layer(selected_layer)


geojson_layer = GeoJSON(
    data=data, name="ROIs", hover_style={"fillColor": "red"}
    )
geojson_layer.on_click(geojson_onclick_handler)
m.add_layer(geojson_layer)
m


Map(bottom=419115.18173217773, center=[34.016474746300055, -120.31488352768683], controls=(ZoomControl(options…

## Add the Marker cluser to the map

In [128]:
from ipyleaflet import LayerGroup
from ipyleaflet import Map, Marker, MarkerCluster
from ipywidgets import HTML

markers = ()
for points_list in points_list_collection:
    for point in points_list:
        marker = Marker(location=point,draggable=False,title=f"Location:({point[1]},{point[0]})")
        message = HTML(value=f"Location:({point[1]},{point[0]})")
        marker.popup = message
        markers=markers + (marker,)

m.add_layer(MarkerCluster(name = "CoastSat Shoreline Data",markers = markers,))

m


Map(bottom=419115.18173217773, center=[34.016474746300055, -120.31488352768683], controls=(ZoomControl(options…

# Add the Selected ROIs a Dictionary 
---
1. Print the ID's of the selected ROIs
2. Add the selected ROIs to the dictionary, selected_ROI
3. Optionally, print the selected_ROI
4. Download the geojson file of the selected_ROI

In [129]:
def get_selected_roi(selected_set:tuple)-> dict:
    """
    Returns the geojson of the ROIs selected by the user

    Arguments:
    -----------
    selected_set:tuple
        A tuple containing the ids of the ROIs selected by the user
    
    Returns:
    -----------
    geojson_polygons: dict
       geojson dictionary containing all the ROIs selected  
    """
    # Check if selected_set is empty
    assert len(selected_set) != 0, "\n Please select at least one ROI from the map before continuing."
    # Create a dictionary for the selected ROIs and add the user's selected ROIs to it  
    selected_ROI={}
    selected_ROI["features"] = [
        feature
        for feature in data["features"]
        if feature["properties"]["id"] in selected_set
    ]
    return selected_ROI

In [130]:
selected_ROI=get_selected_roi(selected_set)
selected_ROI

{'features': [{"geometry": {"coordinates": [[[-120.455436, 34.0365], [-120.455436, 34.0765], [-120.415436, 34.0765], [-120.415436, 34.0365], [-120.455436, 34.0365]]], "type": "Polygon"}, "properties": {"id": 19, "style": {"color": "blue", "fillColor": "grey", "fillOpacity": 0.2, "weight": 2}}, "type": "Feature"},
  {"geometry": {"coordinates": [[[-120.350147, 34.038343], [-120.350147, 34.078343], [-120.310147, 34.078343], [-120.310147, 34.038343], [-120.350147, 34.038343]]], "type": "Polygon"}, "properties": {"id": 29, "style": {"color": "blue", "fillColor": "grey", "fillOpacity": 0.2, "weight": 2}}, "type": "Feature"}]}

### Download the Selected ROI(s) to a geojson file

In [131]:
selected_roi_file="selected_roi.geojson"
write_to_geojson_file(selected_roi_file,selected_ROI,perserve_id=True)

19
29


## Download the Data Selected with CoastSat
---
1. Preview the data available with CoastSat's check_images_available()
2. Download the data available with CoastSat's retrieve_images()
3. Transform the data into jpgs CoastSat's save_jpg()

In [132]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects
import ee
# NOTE: modify the "import gda"l in SDS_download to "from osgeo import gdal" to reslove the module not found error

In [133]:
def get_ids_selected_roi_from_file(selected_roi_file: str)-> list:
    """
     Returns a list of the  ROI ids selected by the user

    Arguments:
    -----------
    selected_roi_file: str
        The filename of the geojson file containing all the ROI selected by the user
    
    Returns:
    -----------
    selected_ids: list
      list of ids for all the ROIs selected by the user  
    """
    with open(selected_roi_file) as f:
        data = geojson.load(f)
    # Get the id from the properties of each rectangle
    selected_ids=[item.get("properties").get("id") for item in data["features"]]
    return selected_ids

In [134]:
def read_selected_roi_from_file(selected_roi_file: str)-> dict:
    """
     Returns the geojson of the selected ROIs from the file specified by selected_roi_file

    Arguments:
    -----------
    selected_roi_file: str
        The filename of the geojson file containing all the ROI selected by the user
    
    Returns:
    -----------
    data: dict
        geojson of the selected ROIs
    """
    with open(selected_roi_file) as f:
        data = geojson.load(f)
    return data

In [137]:
selected_roi_file="selected_roi.geojson"
selected_ROI=read_selected_roi_from_file(selected_roi_file)

### For each item in selected_ROI
1. Create a coastSatBBOX
2. Set Coast's polygon equal to the coastSatBBOX
3. Set all the necessary CoastSat parameters
4. Check images available for download

In [138]:
from datetime import datetime
def generate_datestring():
    """"
    Returns a string in the following format %Y-%m-%d__%H_hr_%M_min.
    EX: "ID02022-01-31__13_hr_19_min"
    """
    date=datetime.now()
    print(date)
    return date.strftime('%Y-%m-%d__%H_hr_%M_min')

In [139]:
def check_images_available_selected_ROI(selected_ROI:dict)->list:
    """"
   
    Return a list of dictionaries containing all the input parameters such as polygon, dates, sat_list, sitename, and filepath. This list can be used
    to retrieve images using coastsat. 
    
    Arguments:
    -----------
    selected_ROI:  dict
        A geojson dictionary containing all the ROIs selected by the user.
   
   Returns:
    -----------
   list_of_inputs: list
        A list of dictionaries containing all the input parameters such as polygon, dates, sat_list, sitename, and filepath
    """
    list_of_inputs=[]; 
    if selected_ROI["features"] != []:
        date_str=generate_datestring()
        for counter, ROI in enumerate(selected_ROI["features"]):
            coastSatBBOX=ROI["geometry"]["coordinates"]
            polygon=coastSatBBOX
            # it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
            polygon = SDS_tools.smallest_rectangle(polygon)
            # date range
            dates = ['2018-12-01', '2019-01-01']
            # satellite missions
            sat_list = ['S2','L7','L8']
            # name of the site
            sitename = 'ID'+str(counter)+date_str
            # directory where the data will be stored
            filepath = os.path.join(os.getcwd(), 'data')
            # put all the inputs into a dictionnary
            inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}
            # before downloading the images, check how many images are available for your inputs
            SDS_download.check_images_available(inputs)
            list_of_inputs.append(inputs)
    return list_of_inputs

In [140]:
class AuthenticationError(Exception):
    """AuthenicationError: raised when an authentication error is raised by Google Earth Engine (EE) 
    Args:
        Exception: Inherits from the base exception class
    """
    def __init__(self, msg="You have not authenticated with Google Earth Engine"):
        self.msg=msg
        super().__init__(self.msg)
    def __str__(self):
        return (f"{self.msg}")
    

In [141]:
def handle_AuthenticationError():
    ee.Authenticate()
    ee.Initialize()

In [142]:
try:
    try:
        list_of_inputs=check_images_available_selected_ROI(selected_ROI)
        list_of_inputs
        if list_of_inputs == []:
            print("Error: No ROIs were selected. Please click a valid ROI on the map")
    except ee.EEException as exception:
        print(exception)
        raise AuthenticationError
except AuthenticationError:
    handle_AuthenticationError()

2022-02-01 16:06:18.001579
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 6 images
  L7: 3 images
  L8: 4 images
  Total: 13 images
- In Landsat Tier 2:
  L7: 1 images
  L8: 4 images
  Total: 5 images
Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 6 images
  L7: 2 images
  L8: 4 images
  Total: 12 images
- In Landsat Tier 2:
  L7: 0 images
  L8: 4 images
  Total: 4 images


In [143]:
list_of_inputs

[{'polygon': [[[-120.455436, 34.0365],
    [-120.415436, 34.0365],
    [-120.415436, 34.0765],
    [-120.455436, 34.0765],
    [-120.455436, 34.0365]]],
  'dates': ['2018-12-01', '2019-01-01'],
  'sat_list': ['S2', 'L7', 'L8'],
  'sitename': 'ID02022-02-01__16_hr_06_min',
  'filepath': 'C:\\1_USGS\\CoastSeg\\repos\\CoastSeg_fork\\Seg2Map\\notebooks\\data'},
 {'polygon': [[[-120.350147, 34.038343],
    [-120.310147, 34.038343],
    [-120.310147, 34.078343],
    [-120.350147, 34.078343],
    [-120.350147, 34.038343]]],
  'dates': ['2018-12-01', '2019-01-01'],
  'sat_list': ['S2', 'L7', 'L8'],
  'sitename': 'ID12022-02-01__16_hr_06_min',
  'filepath': 'C:\\1_USGS\\CoastSeg\\repos\\CoastSeg_fork\\Seg2Map\\notebooks\\data'}]

## Preprocessing the Downloaded Data
---

For each input dictionary in the list_of_inputs that was created by **check_images_available_selected_ROI**
1. Download the .tif and the associated metadata with **SDS_download.retrieve_images**
2. Create the pre_process_settings that determine how jpgs are created
3. Create jpgs from the downloaded .tif file using the metadata & pre_process_settings with **DS_preprocess.save_jpg(metadata, pre_process_settings)**

In [144]:
# for inputs in list_of_inputs:
#     metadata = SDS_download.retrieve_images(inputs)  
for inputs in list_of_inputs:
    print("\ninputs: ",inputs,"\n")
    metadata = SDS_download.retrieve_images(inputs)
    # metadata = SDS_download.get_metadata(inputs)
    pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    # add the inputs defined previously
    'inputs': inputs,
    }
    print(metadata)
    SDS_preprocess.save_jpg(metadata, pre_process_settings)


inputs:  {'polygon': [[[-120.455436, 34.0365], [-120.415436, 34.0365], [-120.415436, 34.0765], [-120.455436, 34.0765], [-120.455436, 34.0365]]], 'dates': ['2018-12-01', '2019-01-01'], 'sat_list': ['S2', 'L7', 'L8'], 'sitename': 'ID02022-02-01__16_hr_06_min', 'filepath': 'C:\\1_USGS\\CoastSeg\\repos\\CoastSeg_fork\\Seg2Map\\notebooks\\data'} 

Images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 6 images
  L7: 3 images
  L8: 4 images
  Total: 13 images
- In Landsat Tier 2:
  L7: 1 images
  L8: 4 images
  Total: 5 images

S2: 6 images
100%
L7: 3 images
100%
L8: 4 images
100%
0 out of 6 Sentinel-2 images were merged (overlapping or duplicate)
{'L7': {'filenames': ['2018-12-04-18-30-31_L7_ID02022-02-01__16_hr_06_min_pan.tif', '2018-12-04-18-30-55_L7_ID02022-02-01__16_hr_06_min_pan.tif', '2018-12-20-18-30-04_L7_ID02022-02-01__16_hr_06_min_pan.tif'], 'acc_georef': [6.105999946594238, 9.026000022888184, 4.283999919891357], 'epsg': [32611, 32610